In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

model_name = "BAAI/bge-base-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

## Sample text test

In [22]:
text = "Your sample text goes here."

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

In [23]:
text2 = "Je m'apelle François"

inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs2)
    embeddings2 = outputs.last_hidden_state.mean(dim=1)

In [ ]:
# Assuming embeddings and embeddings2 are tensors of shape (1, hidden_dim)
cosine_similarity = torch.nn.functional.cosine_similarity(embeddings, embeddings2, dim=1)

# Print the similarity score
print("Cosine Similarity:", cosine_similarity.item())

Cosine Similarity: 0.5604743957519531


## Test love hate

In [28]:
text = "I love you"
text2 = "They hate me"

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs2)
    embeddings2 = outputs.last_hidden_state.mean(dim=1)

# Assuming embeddings and embeddings2 are tensors of shape (1, hidden_dim)
cosine_similarity = torch.nn.functional.cosine_similarity(embeddings, embeddings2, dim=1)

# Print the similarity score
print("Cosine Similarity:", cosine_similarity.item())

Cosine Similarity: 0.5622000098228455


## Samples to actually search

In [6]:
import torch
import os
import numpy as np

# Define a function to vectorize texts
def vectorize_texts(texts, tokenizer, model):
    # Tokenize all texts at once
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    # Get embeddings without gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
        # Calculate mean embedding for each text
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Shape: (n_texts, hidden_dim)
    return embeddings

# List of texts to vectorize
texts = ["¿Cuál es la opinión de Luis con respecto a la automatización de los puestos?", "Luis comenta que la automatización está reemplazando trabajos repetitivos.", "Luis menciona que la automatización afecta la motivación y competencia de los trabajadores.", "Otra cosa"]  # Add all texts here


# Vectorize the texts
text_embeddings = vectorize_texts(texts, tokenizer, model)

# Directory containing .npy vector files
vector_dir = '/mnt/c/Users/luisg/Desktop/STAR/STAR/examples/conversation_sample_2/vectors'

# Calculate cosine similarity for each text against each .npy vector
for npy_file in os.listdir(vector_dir):
    if npy_file.endswith(".npy"):
        # Load the vector from .npy file
        npy_vector = torch.tensor(np.load(os.path.join(vector_dir, npy_file)))
        
        # Ensure npy_vector is 2D for compatibility
        if npy_vector.dim() == 1:
            npy_vector = npy_vector.unsqueeze(0)
        
        # Compute cosine similarity for each text embedding with the current npy vector
        similarities = torch.nn.functional.cosine_similarity(text_embeddings, npy_vector, dim=1)
        
        # Print or store the results
        print(f"Cosine Similarities with {npy_file}: {similarities.tolist()}")

Cosine Similarities with 1.npy: [0.7147541046142578, 0.7099173069000244, 0.7333047389984131, 0.6066607236862183]
Cosine Similarities with 10.npy: [0.7633360624313354, 0.8169689178466797, 0.8212881684303284, 0.6160691380500793]
Cosine Similarities with 11.npy: [0.7434858083724976, 0.7165929079055786, 0.6686182022094727, 0.6310552954673767]
Cosine Similarities with 2.npy: [0.724294126033783, 0.7494648098945618, 0.7736704349517822, 0.6235658526420593]
Cosine Similarities with 3.npy: [0.6889639496803284, 0.7040612697601318, 0.7218436002731323, 0.63972008228302]
Cosine Similarities with 4.npy: [0.7428139448165894, 0.7360765337944031, 0.7158006429672241, 0.610838770866394]
Cosine Similarities with 5.npy: [0.7063431143760681, 0.7119358777999878, 0.7008657455444336, 0.6176100373268127]
Cosine Similarities with 6.npy: [0.7488690614700317, 0.8009746670722961, 0.7647774815559387, 0.6152656674385071]
Cosine Similarities with 7.npy: [0.7615599632263184, 0.7418962717056274, 0.7225896716117859, 0.634

In [8]:
# Calculate cosine similarity for each text against each .npy vector
for npy_file in os.listdir(vector_dir):
    if npy_file.endswith(".npy"):
        # Load the vector from .npy file
        npy_vector = torch.tensor(np.load(os.path.join(vector_dir, npy_file)))
        
        # Ensure npy_vector is 2D for compatibility
        if npy_vector.dim() == 1:
            npy_vector = npy_vector.unsqueeze(0)
        
        # Compute cosine similarity for each text embedding with the current npy vector
        similarities = torch.nn.functional.cosine_similarity(text_embeddings, npy_vector, dim=1)
        
        # Get the number of similarities available (up to 10)
        num_top_values = min(10, similarities.size(0))
        
        # Get top values and indices based on available size
        top_values, top_indices = torch.topk(similarities, num_top_values)
        
        # Print or store the top results
        print(f"Top {num_top_values} Cosine Similarities with {npy_file}:")
        for i, (value, idx) in enumerate(zip(top_values.tolist(), top_indices.tolist())):
            print(f"  {i + 1}: Text '{texts[idx]}' - Similarity: {value}")

Top 4 Cosine Similarities with 1.npy:
  1: Text 'Luis menciona que la automatización afecta la motivación y competencia de los trabajadores.' - Similarity: 0.7333047389984131
  2: Text '¿Cuál es la opinión de Luis con respecto a la automatización de los puestos?' - Similarity: 0.7147541046142578
  3: Text 'Luis comenta que la automatización está reemplazando trabajos repetitivos.' - Similarity: 0.7099173069000244
  4: Text 'Otra cosa' - Similarity: 0.6066607236862183
Top 4 Cosine Similarities with 10.npy:
  1: Text 'Luis menciona que la automatización afecta la motivación y competencia de los trabajadores.' - Similarity: 0.8212881684303284
  2: Text 'Luis comenta que la automatización está reemplazando trabajos repetitivos.' - Similarity: 0.8169689178466797
  3: Text '¿Cuál es la opinión de Luis con respecto a la automatización de los puestos?' - Similarity: 0.7633360624313354
  4: Text 'Otra cosa' - Similarity: 0.6160691380500793
Top 4 Cosine Similarities with 11.npy:
  1: Text '¿Cuál

In [ ]:
# Initialize a list to store all similarity results
all_similarities = []

# Threshold for similarity scores
similarity_threshold = 0.8

# Calculate cosine similarity for each text against each .npy vector
for npy_file in os.listdir(vector_dir):
    if npy_file.endswith(".npy"):
        # Load the vector from .npy file
        npy_vector = torch.tensor(np.load(os.path.join(vector_dir, npy_file)))
        
        # Ensure npy_vector is 2D for compatibility
        if npy_vector.dim() == 1:
            npy_vector = npy_vector.unsqueeze(0)
        
        # Compute cosine similarity for each text embedding with the current npy vector
        similarities = torch.nn.functional.cosine_similarity(text_embeddings, npy_vector, dim=1)
        
        # Store each similarity result that meets the threshold, along with the associated file and text
        for idx, similarity in enumerate(similarities):
            if similarity.item() >= similarity_threshold:
                all_similarities.append((similarity.item(), texts[idx], npy_file))

# Sort results in descending order for readability
all_similarities.sort(reverse=True, key=lambda x: x[0])

# Print matches that meet or exceed the threshold
print(f"Cosine Similarity Matches Across All .npy Files (≥ {similarity_threshold}):")
for similarity, text, npy_file in all_similarities:
    print(f"File '{npy_file}', Text '{text}' - Similarity: {similarity}")

Cosine Similarity Matches Across All .npy Files (≥ 0.8):
File '10.npy', Text 'Luis menciona que la automatización afecta la motivación y competencia de los trabajadores.' - Similarity: 0.8212881684303284
File '10.npy', Text 'Luis comenta que la automatización está reemplazando trabajos repetitivos.' - Similarity: 0.8169689178466797
File '6.npy', Text 'Luis comenta que la automatización está reemplazando trabajos repetitivos.' - Similarity: 0.8009746670722961


In [14]:
# Initialize a set to store unique filenames that meet the threshold
matching_files = set()

# Calculate cosine similarity for each text against each .npy vector
for npy_file in os.listdir(vector_dir):
    if npy_file.endswith(".npy"):
        # Load the vector from .npy file
        npy_vector = torch.tensor(np.load(os.path.join(vector_dir, npy_file)))
        
        # Ensure npy_vector is 2D for compatibility
        if npy_vector.dim() == 1:
            npy_vector = npy_vector.unsqueeze(0)
        
        # Compute cosine similarity for each text embedding with the current npy vector
        similarities = torch.nn.functional.cosine_similarity(text_embeddings, npy_vector, dim=1)
        
        # Check if any similarity meets or exceeds the threshold
        if any(similarity.item() >= similarity_threshold for similarity in similarities):
            # Add the filename without the .npy extension to the set
            matching_files.add(os.path.splitext(npy_file)[0])

# Output the set of matching filenames
print(f"Files with similarity ≥ {similarity_threshold}:", matching_files)

Files with similarity ≥ 0.8: {'10', '6'}


## Generate grok response with these chunks

In [16]:
### INITIAL MESSAGE WITH GROK ###
import sys
sys.path.append('/mnt/c/Users/luisg/Desktop/STAR/STAR/scripts')

# Now you can import grok as if it's in the same directory
import grok

# Example usage
client = grok.initialize_grok_api()

In [ ]:
# Initialize chunk set
chunk_set = matching_files

# Initialize a list to store the content of each chunk
chunk_list = []

# Set chunks directory
chunk_directory = '/mnt/c/Users/luisg/Desktop/STAR/STAR/examples/conversation_sample_2/chunks'

# Read each chunk file and store its content
for chunk_id in chunk_set:
    file_path = os.path.join(chunk_directory, f"{chunk_id}.txt")
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            chunk_content = file.read()
            chunk_list.append(chunk_content)
    except FileNotFoundError:
        chunk_list.append(f"Chunk file {chunk_id}.txt not found.")

['LUIS (11:14.00 - 11:19.00):  Ya está, quería volver al tema de los cuatro grupos. Entonces tenemos los caramelitos,  que serían el grupo muy motivado y muy competente. Tenemos luego un grupo muy competente pero poco motivado  que a eso habría que ver cómo aumentar esa motivación, ¿vale? Y que eso ya se ha visto  que pues por ejemplo para el Sistem se ha aumentado con este incremento de las tecnologías  y viendo que las condiciones laborales son buenas.\nLIDIA (11:38.27 - 11:43.36):  La condición es incremento de herramientas que me permiten adquirir esa motivación desde\nLUIS (11:43.36 - 11:51.03):  otras fuentes. Yo creo que también el reemplazo o la automatización de tareas repetitivas también es un aliciente  porque al final por una carrera muy técnica y un conocimiento muy bueno que tú tengas,  todos los trabajos hay tareas repetitivas, que eso es un coñazo, a nadie le gustan.  Y eso es bueno para poder hacer luego las personas un trabajo inteligente y productivo  y no perder hor

In [20]:
user_question = "¿Cuál es la opinión de Luis con respecto a la automatización de los puestos?"

grok_RAG_response = grok.respond_RAG_question(client, user_question, chunk_list)

print(grok_RAG_response)

Luis ve la automatización de puestos de trabajo como una oportunidad para aumentar la motivación y la productividad de los empleados. Considera que reemplazar tareas repetitivas con máquinas permite a las personas enfocarse en trabajos más inteligentes y productivos, lo cual es un aliciente para los empleados. Además, menciona que la automatización ha afectado y seguirá afectando a más empleos, no solo los repetitivos, sino también aquellos que requieren menos repetición como conductores o trabajos de oficina, lo que puede generar desmotivación. Como solución, sugiere opciones como la renta básica universal para que los afectados puedan estudiar y adquirir nuevas competencias.
